## Ingest required fake data into MYSQL database

In [1]:
from pathlib import Path
import sys

cwd = Path.cwd().parent.joinpath("money_map")
sys.path.append(cwd)

from money_map.ingestion.rbpn_ingestion import RBPN_Ingestor
from money_map.ingestion.categories_ingestion import Categories_Ingestor
from money_map.pipelines.transactions import recreate_transactions_labeled

### Ingest fake RBPN transaction data

In [2]:
# define target path
file_name = "fake_rbpn_data.csv"
rbpn_file_path = Path.cwd().parent.joinpath("data/interim").joinpath(file_name)

# init ingestor
ingestor = RBPN_Ingestor()
ingestor.connector.create_tables()

#ingest data
ingestor.ingest_data(file_path=rbpn_file_path)

-> 182 new rows ingested!


### Ingest categories for labeling transactions

In [3]:
file_name = "fake_categories.json"
file_path = Path.cwd().parent.joinpath("data/interim").joinpath(file_name)

# init
cat_ingestor = Categories_Ingestor()
cat_ingestor.connector.create_tables()

#ingest data
cat_ingestor.ingest_data(file_path=file_path)


 -> Recreated:  transaction_categories


###  Ingest fake labeled data

In [4]:
import pandas as pd
file_name = "fake_rbpn_participants_labeled.csv"
file_path = Path.cwd().parent.joinpath("data/interim").joinpath(file_name)


df = pd.read_csv(file_path,sep=";")
df.head(5)

,row_number,sender_bank_name,sender_iban,receiver_name,receiver_iban,booking_text,purpose_char,category_id,category_1,category_2,category_3
0,1,RBPN,GB87YODA19011530005979,"Adams, Walker and Jones",GB60DSOV30707801799689,ABSCHLUSS,drive question we follow,37,Variable expenses,Living expenses,Fashion
1,2,RBPN,GB87YODA19011530005979,Amy Riley,GB50YYFZ33231044832099,LASTSCHRIFT,style grow eight carry carry,37,Variable expenses,Living expenses,Fashion
2,3,RBPN,GB87YODA19011530005979,Andrew Myers,GB25CZXL23186024805804,DAUERAUFTRAG,tree red present pick paper,16,Fixed expenses,Insurance,Supplementary health insurance
3,4,RBPN,GB87YODA19011530005979,Arnold-Lawrence,GB89PEAD91436481302063,Aufladung Mobilfunkguthaben,environment fear either plant,30,Fixed expenses,Other contracts,Mobile phone
4,6,RBPN,GB87YODA19011530005979,Brian Contreras,GB69OVYM85148454554764,Auszahlung girocard,man board discover likely study case air,49,Variable expenses,Entertainment,Vacation


In [5]:
df.to_sql(con=ingestor.connector.create_sql_engine(),
            name="participants_labeled",
            if_exists="replace",
            index=False)

48

In [6]:
recreate_transactions_labeled(engine=ingestor.connector.create_sql_engine())